## How Cost is Calculated

Welcome! This notebook provides the practical demonstration for our lecture on cost calculation. We've discussed the theory: API usage is priced on a **pay-per-token** basis, and the cost is a function of both the **input tokens** you send and the **output tokens** you receive.

The formula is:
`Total Cost = (Input Tokens × Price per Input Token) + (Output Tokens × Price per Output Token)`

Now, let's see how to apply this in practice. The most accurate way to calculate the cost of a specific API call is to use the `usage` object that is returned in the API's response. This object gives you the precise token counts, taking all guesswork out of the equation.

## Defining the Pricing Structure

First, let's define the pricing for the model we're using, `gpt-4o-mini`. It's a best practice to define these values as constants in your application so you can easily update them if prices change.

> Note: The following prices are for gpt-4o-mini as of August 2025. Always check the official OpenAI pricing page for the most current rates. Prices are typically listed per 1 million tokens.
>

In [ ]:
import litellm
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

print("Libraries and environment variables loaded successfully.")

## Calculating Cost from API Response

Now, we'll make a simple, low-cost API call. Our calculation logic will pull the correct prices from the dictionary we just defined. We'll then inspect the `usage` attribute from the API response to get the exact token counts for our transaction.

This is the ideal method for logging, auditing, and managing costs in a production application, as it's both accurate and scalable.

## Conversation Cost Analysis

Now let's explore how costs accumulate during a multi-turn conversation. Each API call includes the entire conversation history, which means the input token count (and therefore cost) increases with each exchange.

We'll simulate a 4-round conversation to demonstrate this cost escalation pattern.